# OPENCV,mediapipe를 활용한 손가락 관절 인식

In [ ]:
import cv2
import mediapipe as mp

cap=cv2.VideoCapture(0)
mpHands=mp.solutions.hands

hands=mpHands.Hands()
print(hands)

mpDraw=mp.solutions.drawing_utils

while True:
    success,img=cap.read()
    #openCV는 RGB를 BGR로 저장함
    imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results=hands.process(imgRGB)
    print(results.multi_hand_landmarks)
    #손가락을 인식했다면
    if results.multi_hand_landmarks:
        for i in results.multi_hand_landmarks:
            mpDraw.draw_landmarks(img,i)
    cv2.imshow('text',img)
    #1밀리초마다 0xFF비트를 검사해서 q 눌렀으면 종료
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break

# 랜드마크 간 선 그리기

In [ ]:
import cv2
import mediapipe as mp

cap=cv2.VideoCapture(0)
mpHands=mp.solutions.hands
hands=mpHands.Hands()
mpDraw=mp.solutions.drawing_utils

while True:
    success,img=cap.read()
    #이미지 반전하는 함수임, 1=좌우반전, -1=상하반전
    img=cv2.flip(img,1)
    imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results=hands.process(imgRGB)
    
    if results.multi_hand_landmarks:
        for i in results.multi_hand_landmarks:
            mpDraw.draw_landmarks(img,i,mpHands.HAND_CONNECTIONS)
    cv2.imshow('img',img)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break

# 8번 관절 타게팅

In [ ]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
cap=cv2.VideoCapture(0)
my_hands=mp.solutions.hands.Hands()
mpDraw=mp.solutions.drawing_utils

while True:
    success,img=cap.read()
    image=cv2.resize(img,(1920,1080))
    image=cv2.flip(image,1)
    f_height,f_width,idx=image.shape
    rgb_img=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    #손인식 결과
    output=my_hands.process(rgb_img)
    hands=output.multi_hand_landmarks
    if hands:
        for hand in hands:
            mpDraw.draw_landmarks(image,hand)
            landmarks=hand.landmark
            for id, landmark in enumerate(landmarks):
                x=int(landmark.x*f_width)
                y=int(landmark.y*f_height)
                if id==8:
                    cv2.circle(img=image,center=(x,y),radius=20,color=(0,255,0),thickness=10)

    # cv2.imshow('img',image)
    plt.imshow(image)
    plt.axis('off')
    plt.show()
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break


# 클래스 직접구현(잡다한 기능을 붙힐 수 있음. 가장 최신의 손은 초록 점으로)

In [15]:
import cv2
import mediapipe as mp
cap=cv2.VideoCapture(0)
#클래스 직접 만들어서 써보기
class handDetector():
    def __init__(self,mode=False,maxHands=2,modelComplexity=1,detectionCon=0.5,trackCon=0.5):
        self.mode=mode
        self.maxHands=maxHands
        self.modelComplexity=modelComplexity
        self.detectionCon=detectionCon
        self.tranCon=trackCon
        self.mpHands=mp.solutions.hands
        self.hands=self.mpHands.Hands(self.mode,self.maxHands,self.modelComplexity,self.detectionCon,self.tranCon)
        self.mpDraw=mp.solutions.drawing_utils

    def findHands(self,img,draw=True):
        imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        self.results=self.hands.process(imgRGB)
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img,handLms,self.mpHands.HAND_CONNECTIONS)
        return img
    
    def findPosition(self,img,handN=0,draw=True):
        lmList=[]
        if self.results.multi_hand_landmarks:
            mpHands=self.results.multi_hand_landmarks[handN]
            for id, lm in enumerate(mpHands.landmark):
                h,w,c=img.shape
                cx,cy=int(lm.x*w),int(lm.y*h)
                lmList.append([id,cx,cy])#21개 점 위치 저장하기
                if draw:
                    cv2.circle(img,(cx,cy),10,(155,255,0),cv2.FILLED)
        return lmList
detector=handDetector()

while True:
    flag,img=cap.read()
    img=cv2.flip(img,1)

    img=detector.findHands(img)
    lmList=detector.findPosition(img)
    cv2.imshow('img',img)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break



KeyboardInterrupt: 